# LangChain Ollama

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    tags=["latest"],
    temperature=0.8
)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful translator. Translate the user sentence to Middle English."),
    HumanMessage(content="I love programming."),
]

response = llm.invoke(messages)
print(response.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful translator. Translate the user sentence to Middle English."),
    HumanMessage(content="I love programming."),
]

for token in llm.stream(messages):
    print(token.content, end="")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant and subject matter expert in {topic}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm | StrOutputParser()

for token in chain.stream(
    {
        "topic": "Politics",
        "input": "Is Donald Trump a good president of US? as compared to Joe Biden",
    }
):
    print(token, end="")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


system_prompt = "You are a helpful assistant and subject matter expert in {topic}."
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        ("user", "{input}"),
    ]
)

# chain = prompt | llm | StrOutputParser()
prompt = prompt_template.invoke(
    {
        "topic": "Math",
        "input": "what is The Collatz Conjecture?",
    }
)

for token in llm.stream(prompt):
    print(token.content, end="")

# LangGraph Ollama 

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:3b-instruct-q5_K_M",
    # tags=["latest"],
    temperature=0.8
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [ ]:
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        print(event)
        for value in event.values():
            print(value)
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

# LangChain LangGraph Ollama - Build a Chatbot

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

llm = ChatOllama(
    model="qwen2.5:3b-instruct-q5_K_M",
    temperature=0.5
)

In [ ]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="Hi! I'm Bob")])
print(response.content)

In [ ]:
response = llm.stream([HumanMessage(content="What's my name?")])

for chunk in response:
    print(chunk.content, end="")

In [ ]:
from langchain_core.messages import AIMessage

response = llm.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

print(response.content)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    print(f"state[\"messages\"] in call_model: \n{state["messages"]}")
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
import json
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
state = app.get_state(config)
print(f"state in app.get_state(config) is: \n{state}")
output["messages"]  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
state = app.get_state(config)
print(f"state in app.get_state(config) is: \n{state}")
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc234"}}
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    print(f"state in call_model: \n{state}")
    prompt = prompt_template.invoke(state)
    response = llm.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = llm.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=60000,
    strategy="last",
    token_counter=llm.get_num_tokens_from_messages,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = llm.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc789"}}
query = "who are you?"
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")
    # print(chunk.content, end="")

# Web Search

In [6]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

result = search.invoke("who is Obama?")
print(result)

Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). He was the third African American to be elected to that body since the end of Reconstruction (1877). In 2009 he was awarded the Nobel Peace ... Barack Obama, the former Illinois senator and two-term commander-in-chief, has kept busy since he left office in 2017. Barack Obama served as the 44th president of the United States (2009-17) and was the first African American to hold that post. A member of the Democratic Party, Obama previously represented Illinois in the U.S. Senate from 2005 to 2008. He was honoured with the Nobel Peace Prize in 2009. Barack Obama - 44th President, Political Career, Legacy: In 1996 he was elected to the Illinois Senate, where, most notably, he helped pass legislation that tightened campaign finance regulations, 

In [17]:
from langchain_core.messages import HumanMessage, ToolMessage

message = ToolMessage(content="Hi", tool_call_id=123)
message.content.lower()

'hi'

In [ ]:
import json

from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(output_format='json')

search_results = json.loads(search.invoke("Malaysia weather?"))
print(json.dumps(search_results[:2], indent=2))
urls = [item["link"] for item in search_results[:2]]
print(urls)

In [ ]:
import json

from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(output_format='json', backend='news')

result = search.invoke("Current status of Xi Jin Ping visiting to Malaysia")

# for item in result:
#     print(item['link'])

print(json.dumps(json.loads(result), indent=2))

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.messages import AIMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import create_react_agent

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.5
)

tool = DuckDuckGoSearchResults()
agent_chain = create_react_agent(model=llm, tools=[tool])

In [ ]:
user_input = {"messages": [("user", "what is the date today in Malaysia?")]}
result = agent_chain.invoke(user_input)
result["messages"]

# Web Browse Async

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply() 

In [ ]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

In [ ]:
from pprint import pprint

tools_by_name = {tool.name: tool for tool in tools}
pprint(tools_by_name)
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]
extract_text_tool = tools_by_name["extract_text"]

In [ ]:
await navigate_tool.arun(
    {"url": "https://www.msn.com/en-us/money/markets/malaysia-cbank-says-us-tariffs-will-have-impact-but-economy-is-diversified/ar-AA1Cz3n2"}
)

In [ ]:
content = await extract_text_tool.arun(
    {}
)

content

In [ ]:
content = await extract_text_tool.arun(
    {}
)

content

In [ ]:
await get_elements_tool.arun(
    {"selector": "body", "attributes": ["innerText"]}
)

In [ ]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name["current_webpage"].arun({})

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search_tool = DuckDuckGoSearchResults()
print(search_tool.name)
print(search_tool.description)
# all_tools = tools + [search_tool]

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.5
)

llm_with_tools = llm.bind_tools([search_tool])

result = llm_with_tools.invoke("Hi")
print(result)

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()

llm = ChatOllama(
    model="qwen2.5:3b-instruct-q5_K_M",
    temperature=0.5
)

agent_executor = create_react_agent(
    model=llm, 
    tools=[search_tool], 
    checkpointer=memory, 
    prompt="""You are a helpful assistant. Only use tools when necessary. Please think if the given user message require calling tools. If no, just answer the user message directly without calling the tools.""")


In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="How are you?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
import asyncio

urls = ["https://www.accuweather.com/en/my/kuala-lumpur/233776/weather-forecast/233776", "https://www.weather25.com/asia/malaysia?page=today"]

async def scrape_single_url(url):
    try:
        await navigate_tool.ainvoke({"url": url})
        content = await get_elements_tool.ainvoke({"selector": "body"})
        return f"URL: {url}\nContent:\n{content}\n"
    except Exception as e:
        return f"Failed to scrape {url}: {str(e)}"

scraping_tasks = [scrape_single_url(url) for url in urls]
scraped_contents = await asyncio.gather(*scraping_tasks)

result = "\n\n".join(scraped_contents)
result

# Web Search + Langgraph

In [7]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.agents import Tool

ddg_search = DuckDuckGoSearchResults()
# print(ddg_search.description)


In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. Before using the tools, think carefully if you really need to call the tool to form a response. If you don't need the tool, just proceed to generate response without calling the tool.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [9]:
from typing import Annotated

from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

llm = ChatOllama(
    model="qwen2.5:3b-instruct-q5_K_M",
    # model='qwen3:4b',
    temperature=0.5
)
llm_with_tools = llm.bind_tools([ddg_search])


def chatbot(state: State):
    prompt = prompt_template.invoke(state)
    # print(prompt)
    # response = llm.invoke(prompt)
    return {"messages": [llm_with_tools.invoke(prompt)]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[ddg_search])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [ ]:
# user_input = "Hi there! My name is Will."

# # The config is the **second positional argument** to stream() or invoke()!
# events = graph.stream(
#     {"messages": [{"role": "user", "content": user_input}]},
#     config,
#     stream_mode="values",
# )
# for event in events:
#     event["messages"][-1].pretty_print()

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [ ]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

# Search and Scrape

In [ ]:
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from typing import Optional
from langchain_core.tools.base import ArgsSchema
from langchain_core.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from pydantic import BaseModel, Field
import json


class SearchAndScrapeToolInput(BaseModel):
    query: str = Field(description="Web search query string retrieved from user input")

# Setup search tool
search_tool = DuckDuckGoSearchResults(output_format='json')

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply() 

# Setup Playwright tools
async_browser = create_async_playwright_browser()
playwright_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
navigate_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["navigate_browser"]
get_elements_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["get_elements"]

# Define your custom tool
class SearchAndScrapeTool(BaseTool):
    name: str = "search_and_scrape_websites"
    description: str = "Use this to search for any online and latest info. Input should be a search query string."
    return_direct: bool = True
    args_schema: Optional[ArgsSchema] = SearchAndScrapeToolInput

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # Step 1: Search websites
        search_results = json.loads(search_tool.invoke(query))
        urls = [item["link"] for item in search_results[:2]]  # Only take top 2

        # Step 2: Scrape websites
        scraped_contents = []
        for url in urls:
            try:
                navigate_tool.invoke({"url": url})
                content = get_elements_tool.invoke({"selector": "body"})  # Scrape page body
                scraped_contents.append(f"URL: {url}\nContent:\n{content}\n")
            except Exception as e:
                scraped_contents.append(f"Failed to scrape {url}: {str(e)}")

        return "\n\n".join(scraped_contents)

    async def _arun(self, query: str):
        raise NotImplementedError("Async not implemented for this simple tool.")

# Instantiate
tool = SearchAndScrapeTool()
tools = [tool]

In [ ]:
# from langchain_core.tools import tool
# from langchain_community.tools import DuckDuckGoSearchResults
# from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
# from langchain_community.tools.playwright.utils import create_async_playwright_browser

# # Setup search tool
# search_tool = DuckDuckGoSearchResults()

# # This import is required only for jupyter notebooks, since they have their own eventloop
# import nest_asyncio

# nest_asyncio.apply() 

# # Setup Playwright tools
# async_browser = create_async_playwright_browser()
# playwright_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
# navigate_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["navigate_browser"]
# get_elements_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["get_elements"]


# @tool
# def search_and_srape_web(query: str) -> str:
#     """Use this to search the web for any online and latest info. Input should be a search query string."""
#     # Step 1: Search websites
#     search_result = search_tool.invoke({"query": query})
#     urls = [r["href"] for r in search_result["results"][:2]]  # Only take top 2

#     # Step 2: Scrape websites
#     scraped_contents = []
#     for url in urls:
#         try:
#             navigate_tool.invoke({"url": url})
#             content = get_elements_tool.invoke({"selector": "body"})  # Scrape page body
#             scraped_contents.append(f"URL: {url}\nContent:\n{content}\n")
#         except Exception as e:
#             scraped_contents.append(f"Failed to scrape {url}: {str(e)}")

#     return "\n\n".join(scraped_contents)

# tools = [search_and_srape_web]

In [ ]:
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from typing import Optional
import asyncio
from langchain_core.tools.base import ArgsSchema
from langchain_core.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from pydantic import BaseModel, Field
import json


class SearchAndScrapeToolInput(BaseModel):
    query: str = Field(
        description=(
            "A concise web search query reformulated from the user's input. "
            "It should include key nouns and entities (e.g. topics, locations) "
            "and temporal terms (e.g. 'today', 'this week') if mentioned. "
            "Omit unnecessary words like 'how', 'is', 'what', etc. "
            "Example: From 'How is the status of Ukraine-Russia conflict today?', "
            "extract 'Ukraine-Russia conflict status today'."
        )
    )


# Setup search tool
search_tool = DuckDuckGoSearchResults(output_format='list', backend='news')

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply() 

# Setup Playwright tools
async_browser = create_async_playwright_browser()
playwright_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
navigate_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["navigate_browser"]
get_elements_tool = {tool.name: tool for tool in playwright_toolkit.get_tools()}["get_elements"]

class SearchAndScrapeToolAsync(BaseTool):
    name: str = "search_and_scrape_websites_async"
    description: str = "Use this to search for any latest news if the news is out of your scope within your knowledge. Input should be a search query string."
    return_direct: bool = True
    args_schema: Optional[ArgsSchema] = SearchAndScrapeToolInput

    def _run(self, query: str):
        raise NotImplementedError("Use async version only.")

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        # Step 1: Search
        search_results = await search_tool.ainvoke(query)
        urls = [item["link"] for item in search_results]
        # Step 2: Scrape in parallel
        async def scrape_single_url(url):
            try:
                await navigate_tool.ainvoke({"url": url})
                content = await get_elements_tool.ainvoke({"selector": "body", "attributes": ["innerText"]})
                content = json.loads(content)
                print(f"Print statement - Content: \n{content}")
                return f"URL: {url}\nContent:\n{content}\n"
            except Exception as e:
                return f""
                # return None

        scraping_tasks = [scrape_single_url(url) for url in urls]
        scraped_contents = await asyncio.gather(*scraping_tasks)

        return "\n\n".join(scraped_contents)

# Instantiate
tool = SearchAndScrapeToolAsync()
tools = [tool]


In [ ]:
from langchain_ollama import ChatOllama


llm = ChatOllama(
    model="qwen2.5:3b-instruct-q5_K_M",
    temperature=0.5
)
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is the impact of the newly implemented US tariff on Malaysia?"

messages = [HumanMessage(query)]

ai_msg = await llm_with_tools.ainvoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
    tool_msg = await tool.ainvoke(tool_call)
    messages.append(tool_msg)

for message in messages:
    message.pretty_print()

In [ ]:
messages

In [ ]:
result = llm_with_tools.invoke(messages)
result.pretty_print()

# Langgraph Tool Call Chatbot (decoupled llm with tools and single llm)

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# tavily_api_key = os.environ["TAVILY_API_KEY"]

True

In [2]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=False,
)

llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen2.5:3b-instruct-q5_K_M',
    temperature=0.5
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "who is the current president of US?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

who is the current president of US?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (8d00be7f-1b5a-468a-8062-ac8e760997df)
 Call ID: 8d00be7f-1b5a-468a-8062-ac8e760997df
  Args:
    query: current president USA
================================= Tool Message =================================
Name: tavily_search

{"query": "current president USA", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Presidents, vice presidents, and first ladies - USAGov", "url": "https://www.usa.gov/presidents", "content": "The president of the United States is the: U.S. head of state; Chief executive of the federal government; Commander-in-Chief of the armed forces; Current president. The 47th and current president of the United States is Donald John Trump. He was sworn into office on January 20, 2025. Former U.S. presidents. The Unit

In [16]:
from typing import Annotated

from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama
from langchain_tavily import TavilySearch

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

# wrapper = DuckDuckGoSearchAPIWrapper(time="d")
# ddg_search = DuckDuckGoSearchResults(
#     api_wrapper=wrapper,
#     output_format='string',
#     backend='text'
# )

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=False,
)

# tools = [ddg_search]
tools = [tavily_search_tool]

llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen2.5:3b-instruct-q5_K_M',
    temperature=0.5
)
llm_with_tools = llm.bind_tools(tools)


def chatbot_with_tools(state: State):
    print("Chatbot with tools is executing....\n")
    print("State for chatbot with tools:\n")
    for message in state["messages"]:
        message.pretty_print()
    print("\nEnd of state for chatbot with tools:\n")
    response = llm_with_tools.invoke(state["messages"])
    print(f"Response from chatbot with tools: {response} \n")
    print(f"Tool calls from chatbot with tools: {response.tool_calls} \n")
    print("Chatbot with tools done executing\n")
    if hasattr(response, "tool_calls") and len(response.tool_calls) <= 0:
        # return {"messages": [AIMessage(content="")]}
        return None
    return {"messages": [response]}

def chatbot(state: State):
    # print(f"\nState: {state["messages"]}")
    print("Chatbot is executing....\n")
    print("State for chatbot:\n")
    for message in state["messages"]:
        message.pretty_print()
    print("End of state for chatbot:\n")
    print("Chatbot done executing\n")
    return {"messages": [llm.invoke(state["messages"])]}

def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the tool node if the last message
    has tool calls. Otherwise, route to the chatbot.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "chatbot"

graph_builder.add_node("chatbot_with_tools", chatbot_with_tools)
graph_builder.add_node("chatbot", chatbot)


# tool_node = ToolNode(tools=[ddg_search])
tool_node = ToolNode(tools=[tavily_search_tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot_with_tools",
    # tools_condition,
    route_tools
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot_with_tools")
# graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [20]:
ddg_search.description

'A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [17]:
def stream_graph_updates(user_input: str):
    for msg, metadata in graph.stream({"messages": [{"role": "user", "content": user_input}]}, stream_mode="messages"):
        if msg.content and metadata["langgraph_node"] == "chatbot":
        # if msg.content:
            print(msg.content, end="", flush=True)


# def stream_graph_updates(user_input: str):
#     for msg, metadata in graph.stream({"messages": [{"role": "user", "content": user_input}]}, stream_mode="messages"):
#         if isinstance(msg, BaseMessage) and hasattr(msg, "content"):
#             print(msg.content or "", end="", flush=True)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Chatbot with tools is executing....

State for chatbot with tools:

================================ Human Message =================================

Hi

End of state for chatbot with tools:

Response from chatbot with tools: content='Hello! How can I assist you today?' additional_kwargs={} response_metadata={'model': 'qwen2.5:3b-instruct-q5_K_M', 'created_at': '2025-05-07T14:23:54.3903461Z', 'done': True, 'done_reason': 'stop', 'total_duration': 715515900, 'load_duration': 43681500, 'prompt_eval_count': 952, 'prompt_eval_duration': 242793100, 'eval_count': 10, 'eval_duration': 421977800, 'message': Message(role='assistant', content='Hello! How can I assist you today?', images=None, tool_calls=None)} id='run-47acb9f9-3737-4d56-b0ae-3020cbb206af' usage_metadata={'input_tokens': 952, 'output_tokens': 10, 'total_tokens': 962} 

Tool calls from chatbot with tools: [] 

Chatbot with tools done executing

Chatbot is executing....

State for chatbot:

================================ Human Me

In [ ]:
from langchain_ollama import ChatOllama

ddg_search = DuckDuckGoSearchResults()
tools = [ddg_search]
llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen3:4b',
    temperature=0.5
)
llm_with_tools = llm.bind_tools(tools)

# llm = ChatOllama(model="llama3.2:3b", temperature=0).bind_tools([])
for chunk in llm_with_tools.stream("Tell me a joke"):
    print(chunk.content, end="|", flush=True)

# Langgraph Tool Call Chatbot (only  llm with tools)

In [ ]:
from typing import Annotated


from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


ddg_search = DuckDuckGoSearchResults()
tools = [ddg_search]
llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen2.5:3b-instruct-q5_K_M',
    temperature=0.5
)
llm_with_tools = llm.bind_tools(tools)


def chatbot_with_tools(state: State):
    # for message in state["messages"]:
    print(f"State: {state["messages"]} \n")
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# def chatbot(state: State):
#     # print(f"\nState: {state["messages"]}")
#     for message in state["messages"]:
#         message.pretty_print()
#     return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot_with_tools", chatbot_with_tools)
# graph_builder.add_node("chatbot", chatbot)


tool_node = ToolNode(tools=[ddg_search])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot_with_tools",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot_with_tools")
graph_builder.set_entry_point("chatbot_with_tools")
graph = graph_builder.compile()

In [13]:
def stream_graph_updates(user_input: str):
    for msg, metadata in graph.stream({"messages": [{"role": "user", "content": user_input}]}, stream_mode="messages"):
        # if msg.content and metadata["langgraph_node"] == "chatbot_with_tools":
        if msg.content:
            print(msg.content, end="", flush=True)


# def stream_graph_updates(user_input: str):
#     for msg, metadata in graph.stream({"messages": [{"role": "user", "content": user_input}]}, stream_mode="messages"):
#         if isinstance(msg, BaseMessage) and hasattr(msg, "content"):
#             print(msg.content or "", end="", flush=True)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

State: [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='df099224-4661-447f-8e2d-7da28f3c9e51')] 

snippet: Hello. I'm Catherine. Rob Hello. I'm Rob. Catherine We both started with what is probably the best-known greeting in English and one of the first words English language students learn, and that is ..., title: BBC Learning English - 6 Minute English / Hello, hello, link: https://www.bbc.com/learningenglish/english/features/6-minute-english/ep-180301, snippet: Customs and culture for saying hello. The origins for some of these ways to say "hello" are fascinating. First, hello was an alternate of the word hallo, which was an alternate of holla. Holla was once used in 14th century England to get someone to stop, but it is now an informal greeting as well. Howdy has history, too!, title: Different Ways to Say Hello in English and How to Respond - Duolingo Blog, link: https://blog.duolingo.com/hello-in-english/, snippet: Hello, my colleague. - This professiona

In [ ]:
from langchain_ollama import ChatOllama

ddg_search = DuckDuckGoSearchResults()
tools = [ddg_search]
llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen3:4b',
    temperature=0.5
)
llm_with_tools = llm.bind_tools(tools)

# llm = ChatOllama(model="llama3.2:3b", temperature=0).bind_tools([])
for chunk in llm_with_tools.stream("Tell me a joke"):
    print(chunk.content, end="|", flush=True)

# Langchain Agent streaming

In [ ]:
from langgraph.prebuilt import create_react_agent

model = ChatOllama(
    model="llama3.2:3b",
    # model='qwen3:4b',
    temperature=0.5
)
agent_executor = create_react_agent(model, tools)

In [ ]:
from langchain_core.messages import HumanMessage

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START


class State(TypedDict):
    topic: str
    joke: str

In [ ]:
llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen3:4b',
    temperature=0.5
)

def generate_joke(state: State):
    llm_response = llm.invoke(
        [
            {"role": "user", "content": f"Generate a joke about {state['topic']}"}
        ]
    )
    return {"joke": llm_response.content}


graph = (
    StateGraph(State)
    # .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "generate_joke")
    # .add_edge("refine_topic", "generate_joke")
    .compile()
)

for message_chunk, metadata in graph.stream(
    {"topic": "ice cream"},
    stream_mode="messages",
):
    if message_chunk.content:
        print(message_chunk.content, end="|", flush=True)

# Back to basic tool calling - testing streaming

In [20]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    # model="llama3.2:3b",
    model='qwen2.5:3b-instruct-q5_K_M',
    temperature=0.5
)

In [21]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [26]:
from langchain_core.messages import HumanMessage, AIMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'fe01501d-a4cf-4404-a428-1d4438d636db', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'e7e3b3e8-2c69-4755-9ed9-2e22d21ed8cb', 'type': 'tool_call'}]


In [23]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b-instruct-q5_K_M', 'created_at': '2025-05-04T09:56:36.6614518Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5338097200, 'load_duration': 3121257700, 'prompt_eval_count': 238, 'prompt_eval_duration': 511141700, 'eval_count': 53, 'eval_duration': 1697613700, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3ee4290b-994f-4a45-863b-90a25f683b25-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '3df41bb5-e131-4272-a6a9-6e3dde6854d4', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '604953ff-351e-4548-9be9-07cfe6f8cc04', 'type': 'tool_call'}], usage_metadata={'input_tokens': 238, 'output_tokens': 53, 'total_tokens': 291}),
 ToolMessage(content='36', name='multiply', tool_call_id='3df41bb5-e131-4272-a6a9-6e3d

In [30]:
messages = [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
            AIMessage(content='')]
messages

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={})]

In [36]:
response = llm.stream(messages)
for chunk in response:
    print(chunk.content, end="", flush=True)

response = llm.invoke(messages)
print(f"\n{type(response)}")

Hello! How can I assist you today? Feel free to ask me any questions or start a conversation on topics that interest you. Whether it's general knowledge, fun facts, or if you need help with something specific, just let me know.
<class 'langchain_core.messages.ai.AIMessage'>


# RAG part 1

In [1]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

llm = ChatOllama(
    model="qwen2.5:3b-instruct",
    temperature=0.5,
)

embedding_model = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [2]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding_model)

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [4]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [6]:
document_ids = vector_store.add_documents(documents=all_splits)

print(len(document_ids))
print(document_ids)

63
['c1f853ea-10df-49fd-af74-8680bbfd8707', '1720e2db-9a84-42d4-9e02-74eb689a545e', 'e266367f-b659-4ec5-8920-d00bc52dba31', '3842e83a-46f5-41ca-9a8c-df2b9be5ce23', '5f559d0a-6c60-4082-acf1-982be146687e', 'be424417-b6b6-4dd8-92a8-c9873387328f', 'c0f09fc0-93bf-4f7a-9a94-c25dbf5caddf', '2d153410-d2b7-42b6-a360-28e827ceb87e', '8a45e35f-1a8b-43d5-9d55-33c13e668fed', 'e9b1dea8-4125-45ad-8681-5eff02931fd3', '49d7b782-6ae2-4e7f-95a7-062f8cb921e9', '3098cd87-546a-43e1-a474-f18c2ede3644', '304e76d8-ff07-4bcc-91ba-9fc1c4c5a5fa', '1133987b-3f1c-496d-86a1-1fd76f5b100d', 'c9324b68-3fcb-4217-81bc-fee356b931ae', 'e8a98617-f85c-4ccf-8331-fdd9f4006a9d', '2c837f1a-aee5-42ee-b42f-45f314b2a170', '002e9332-097a-4569-ba15-a9f5241b4567', '2234563a-c71d-429b-98a5-d3473e5c1ae9', 'ed73074d-c19b-4589-8b21-e231b79f023f', 'ca250a95-6d0b-4ce2-b77b-b2de3475e7f5', 'f1c5eefe-c415-48c8-9c48-0711c601dcd7', 'caa5d0e1-1f71-409a-bda7-ba3a68014249', '2c11a719-bfaf-462b-86ad-4620e913a8af', 'a5bae790-748b-44ad-88e9-8d4dd867e37

In [7]:
from langchain import hub

# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

c:\Users\jayke\Desktop\Documents\Programming\Side Projects\genai-chatbot-project\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [8]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [9]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [10]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [11]:
result = graph.invoke({"question": "What is Task Decomposition?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='e266367f-b659-4ec5-8920-d00bc52dba31', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1638}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth

In [12]:
for step in graph.stream(
    {"question": "What is Task Decomposition?"}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

{'retrieve': {'context': [Document(id='e266367f-b659-4ec5-8920-d00bc52dba31', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1638}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can

In [15]:
for message, metadata in graph.stream(
    {"question": "What is Task Decomposition?"}, stream_mode="messages"
):
    print(message.content, end="|")

Task| Decom|position| refers| to| breaking| down| complex| tasks| into| smaller|,| more| manageable| sub|tasks|.| This| approach| helps| in| understanding| and| executing| complicated| instructions| by| guiding| models| to| think| step|-by|-step| and| decom|pose| problems| into| multiple| steps| or| sub|goals|.||

In [16]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [17]:
total_documents = len(all_splits)
third = total_documents // 3

for i, document in enumerate(all_splits):
    if i < third:
        document.metadata["section"] = "beginning"
    elif i < 2 * third:
        document.metadata["section"] = "middle"
    else:
        document.metadata["section"] = "end"


all_splits[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 8,
 'section': 'beginning'}

In [32]:
all_splits[55]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 37831, 'section': 'end'}, page_content='"content": "Please now remember the steps:\\n\\nThink step by step and reason yourself to the right decisions to make sure we get it right.\\nFirst lay out the names of the core classes, functions, methods that will be necessary, As well as a quick comment on their purpose.\\n\\nThen you will output the content of each file including ALL code.\\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such that\\nFILENAME is the lowercase file name including the file extension,\\nLANG is the markup code block language for the code\'s language, and CODE is the code:\\n\\nFILENAME\\n```LANG\\nCODE\\n```\\n\\nPlease note that the code should be fully functional. No placeholders.\\n\\nYou will start with the \\"entrypoint\\" file, then go to the ones that are imported by that file, and so on.\\nFollow a l

In [21]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding_model)
_ = vector_store.add_documents(all_splits)

In [22]:
from typing import Literal

from typing_extensions import Annotated


class Search(TypedDict):
    """Search query."""

    query: Annotated[str, ..., "Search query to run."]
    section: Annotated[
        Literal["beginning", "middle", "end"],
        ...,
        "Section to query.",
    ]

In [26]:
class State(TypedDict):
    question: str
    query: Search
    context: List[Document]
    answer: str


def analyze_query(state: State):
    structured_llm = llm.with_structured_output(Search)
    query = structured_llm.invoke(state["question"])
    return {"query": query}


def retrieve(state: State):
    query = state["query"]
    retrieved_docs = vector_store.similarity_search(
        query["query"],
        filter=lambda doc: doc.metadata.get("section") == query["section"],
    )
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([analyze_query, retrieve, generate])
graph_builder.add_edge(START, "analyze_query")
graph = graph_builder.compile()

In [33]:
for step in graph.stream(
    {"question": "What does the end of the post say about Task Decomposition? think carefully of which section to retrieve"},
    stream_mode="updates",
):
    print(f"{step}\n\n----------------\n")

{'analyze_query': {'query': {'query': 'What does the end of the post say about Task Decomposition?', 'section': 'end'}}}

----------------

{'retrieve': {'context': [Document(id='8f8ebab5-f36b-4d94-8342-b90928c769e0', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 32858, 'section': 'end'}, page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:'), Document(id='e5c4f739-569c-44ca-a35b-f7dceddf7488', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 30868, 'section': 'end'}, page_content='Here are a sample conversation for task clarification sent to OpenAI ChatCompletion endpoint used by GPT-Engineer. The user inputs are wrapped in {{user input text}}.\n[\n  {\n    "role": "system",\n    "content": "You will read instructions and not carry them out, only seek to clarify them.\\nSpecifically you will first summarise a